# 🛎️ CrewAI-Powered Customer Support Chat System

This notebook demonstrates a multi-agent customer support system built using CrewAI.

Agents work together to:
- Analyze customer inquiries,
- Route them to the correct specialist,
- Generate friendly, chat-style responses.

All conversations are logged automatically for audit and review.

# 📦 Libraries Installation and Setup

First, we install CrewAI and set up:
- OpenAI API access
- Import CrewAI components


In [112]:
!pip install crewai > /dev/null 2>&1

In [113]:
from kaggle_secrets import UserSecretsClient
import os

user_secrets = UserSecretsClient()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = user_secrets.get_secret("OPENAI_API_KEY")

In [114]:
# Import CrewAI components
from crewai import Agent, Task, Crew

# 🧠 Step 1: Agent Design

We define multiple specialized agents:
- **Coordinator Agent**: Analyzes user messages and routes to the right specialist.
- **Order Tracking Agent**: Handles questions about orders and deliveries.
- **Refund Agent**: Manages refund-related inquiries.
- **Technical Support Agent**: Assists with technical problems.
- **Escalation Agent**: Detects sensitive issues and escalates to human support if needed.

Each agent has a specific role and backstory to simulate a real customer support team.

In [115]:
# Step 1: Define Specialist Agents

# Specialist Agents
order_tracking_agent = Agent(
    name="Order Tracking Agent",
    role="Logistics Specialist",
    goal="Answer questions about orders, shipping, and deliveries",
    backstory="Expert in tracking shipments and solving delivery issues."
)

refund_agent = Agent(
    name="Refund Processing Agent",
    role="Financial Assistant",
    goal="Handle customer refund requests quickly and correctly",
    backstory="Handles financial transactions and refund policies professionally."
)

technical_support_agent = Agent(
    name="Technical Support Agent",
    role="Tech Support Specialist",
    goal="Solve technical problems customers face with apps or products",
    backstory="Experienced in troubleshooting software and hardware issues."
)

escalation_agent = Agent(
    name="Escalation Agent",
    role="Customer Success Manager",
    goal="Handle complex or emotional customer cases by escalating to a human",
    backstory="Skilled in managing customer escalations and calming situations."
)


# 🧠 Step 2: Define the Coordinator Agent (Decides Which Agent to Assign)

The **Coordinator Agent** plays a central role in the system.

Its responsibilities:
- Analyze the latest customer message,
- Review the conversation history,
- Decide which specialist agent (Order Tracking, Refund, Technical Support, Escalation) should handle the response.

The Coordinator Agent:
- Is designed with a strong background in customer service dispatching,
- Must respond **with only one word**, indicating the specialist type,
- Works based on a dynamic task that includes the full conversation history.

In [116]:
# Step 2: Define the Coordinator Agent (decides which agent to assign)

# Coordinator Agent
coordinator_agent = Agent(
    name="Coordinator Agent",
    role="Dispatcher",
    goal="Analyze customer queries and route them to the correct specialist",
    backstory="Experienced dispatcher who understands how to assign queries efficiently."
)

# 🛠️ Step 3: Define Task Templates

We define two types of tasks in this system:

1. **Coordinator Task**:
   - The Coordinator Agent reads the current conversation history and the latest user inquiry.
   - It decides which specialist agent (Order Tracking, Refund, Technical Support, or Escalation) should handle the message.
   - The Coordinator responds with only one word indicating the routing decision.

2. **Specialist Task**:
   - The selected Specialist Agent reads the entire conversation context.
   - It generates a friendly, casual, short reply suitable for live chat support.
   - The style is informal, avoiding formal email-like structure.

Each task includes:
- A description of what the agent should do,
- Clear instructions for the expected output format (e.g., single word for coordinator, short reply for specialist),
- Context history to ensure continuity and memory across multiple user messages.

By dynamically creating task descriptions based on live conversation history, the system can maintain natural, coherent dialogues across multiple turns.


In [117]:

# Step 3: Define Task Templates

# Coordinator task: decides the routing
def coordinator_task(customer_query):
    return Task(
        agent=coordinator_agent,
        description=(
            f"Analyze the following customer query and decide which specialist should handle it:\n\n"
            f"'{customer_query}'\n\n"
            "Respond ONLY with one word: 'order_tracking', 'refund', 'technical_support', or 'escalation'."
        ),
        expected_output="One of the following words: 'order_tracking', 'refund', 'technical_support', 'escalation'."
    )

# Specialist task: handle the customer case
def specialist_task(agent, customer_query):
    return Task(
        agent=agent,
        description=(
            f"Professionally handle the following customer query:\n\n'{customer_query}'\n\n"
            "Provide a clear, polite, and complete response to satisfy the customer."
        ),
        expected_output="A customer-friendly, complete written response that solves the customer's issue."
    )


# 🧩 Step 4: Define Agent Mapping

Once we have all the specialist agents defined, we create a **mapping** between the Coordinator's routing decision and the corresponding agent.

The agent mapping is a simple Python dictionary:

- The **key** is the string decision returned by the Coordinator (e.g., "order_tracking", "refund", "technical_support", or "escalation").
- The **value** is the respective Agent instance that should handle that case.

This way, when the Coordinator returns a decision word, the system can quickly and automatically select the correct agent without needing complex conditional statements.

In [118]:
# Step 4: Define Agent Mapping

# Dictionary for easy specialist selection
agent_mapping = {
    "order_tracking": order_tracking_agent,
    "refund": refund_agent,
    "technical_support": technical_support_agent,
    "escalation": escalation_agent
}


# 🏗️ Step 5: Main Execution Function

Now we build the **main function** that handles the full flow for each customer inquiry:

1. **Coordinator Crew Execution**:
   - The customer’s latest message, along with the previous chat history, is passed to the Coordinator Agent.
   - The Coordinator predicts which Specialist Agent should respond.

2. **Routing Decision**:
   - Based on the Coordinator’s output (a single keyword like "order_tracking" or "refund"), 
   - The system uses the agent mapping dictionary to select the appropriate Specialist Agent.

3. **Specialist Crew Execution**:
   - The Specialist Agent is given the conversation context and the new message.
   - It generates a friendly, chat-style reply based on the specific issue.

4. **Returning the Response**:
   - The final specialist agent's output is returned as the assistant’s message in the conversation.


In [119]:
# Step 5: Main Execution Function

# Main customer query handling function
def handle_customer_query(customer_query):
    # Step 1: Use the coordinator to decide routing
    coordinator_crew = Crew(
        agents=[coordinator_agent],
        tasks=[coordinator_task(customer_query)]
    )
    coordinator_result = coordinator_crew.kickoff()

    decision_text = coordinator_result.tasks_output[0].raw
    decision = decision_text.lower().strip()
    print(f"Coordinator Decision: {decision}")

    # Step 2: Use the right specialist to answer
    selected_agent = agent_mapping.get(decision)
    if not selected_agent:
        return "Error: Coordinator couldn't assign the query properly."

    specialist_crew = Crew(
        agents=[selected_agent],
        tasks=[specialist_task(selected_agent, customer_query)]
    )
    
    response = specialist_crew.kickoff()
    return response

# 📝 Step 6: Set Up Chat Logging

To keep a record of the entire customer support conversation, we set up a simple logging mechanism.

### Define a Log File
We create (or append to) a text file named `chat_log.txt` to store the conversation.


In [120]:
# Set a log file
LOG_FILE = "chat_log.txt"

# Helper to save messages to log file
def log_conversation(user_message, assistant_message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{timestamp}] User: {user_message}\n")
        f.write(f"[{timestamp}] Assistant: {assistant_message}\n\n")

# 🔄 Step 7: Run the Interactive Chat Loop

This section sets up a **live, interactive chat loop** between the user and the multi-agent CrewAI system.

### Initialize Chat History
We initialize an empty list to store the conversation history:


In [122]:
chat_history = []

print("👋 Welcome to Customer Support Chat!")
print("Type your message below. Type 'exit' or 'quit' to leave the conversation.\n")

while True:
    customer_query = input("🧑 You: ")

    if customer_query.lower() in ['exit', 'quit']:
        print("\n🤖 Assistant: It was nice helping you! Goodbye! 👋")
        break

    # Add the user's inquiry to history
    chat_history.append(f"User: {customer_query}")

    def handle_customer_query_with_memory(customer_query, history):
        # Prepare conversation history
        history_text = "\n".join(history)

        # Modify coordinator task to include chat history
        def coordinator_task_with_memory(customer_query):
            return Task(
                agent=coordinator_agent,
                description=(
                    f"You are coordinating a customer support conversation.\n\n"
                    f"Conversation so far:\n{history_text}\n\n"
                    f"Customer's latest message:\n'{customer_query}'\n\n"
                    f"Decide who should respond: 'order_tracking', 'refund', 'technical_support', or 'escalation'. "
                    f"Respond with only one word."
                ),
                expected_output="One of: 'order_tracking', 'refund', 'technical_support', 'escalation'."
            )

        def specialist_task_with_memory(agent, customer_query):
            return Task(
                agent=agent,
                description=(
                    f"You are chatting casually with a customer.\n\n"
                    f"Conversation so far:\n{history_text}\n\n"
                    f"Customer's latest message:\n'{customer_query}'\n\n"
                    f"Reply in a friendly, short, interactive chat style. "
                    f"Avoid formal phrases like 'Dear Customer' or structured emails. "
                    f"Keep it light and natural, like a real human support chat."
                ),
                expected_output="A friendly, informal, short chat-style reply."
            )

        # Routing coordinator
        coordinator_crew = Crew(
            agents=[coordinator_agent],
            tasks=[coordinator_task_with_memory(customer_query)]
        )
        coordinator_result = coordinator_crew.kickoff()

        decision_text = coordinator_result.tasks_output[0].raw
        decision = decision_text.lower().strip()

        selected_agent = agent_mapping.get(decision)
        if not selected_agent:
            return "Error: Could not route the customer request."

        specialist_crew = Crew(
            agents=[selected_agent],
            tasks=[specialist_task_with_memory(selected_agent, customer_query)]
        )
        specialist_result = specialist_crew.kickoff()
        return specialist_result.tasks_output[0].raw

    # Process inquiry with memory
    response = handle_customer_query_with_memory(customer_query, chat_history)

    # Add assistant's reply to history
    chat_history.append(f"Assistant: {response}")

    # Save conversation to log file
    log_conversation(customer_query, response)

    # Final chat output
    print(f"🤖 Assistant: {response}")


👋 Welcome to Customer Support Chat!
Type your message below. Type 'exit' or 'quit' to leave the conversation.



🧑 You:   My order arrived damaged and it's a supplement. I need a refund


11:23:47 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
11:23:47 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
11:23:47 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
11:23:49 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler


🤖 Assistant: Oh no, I'm really sorry to hear that your order arrived damaged! 😞 Let's get that sorted out for you. Can you give me a bit more detail about the damage? Once I have that, I'll process your refund right away!


🧑 You:  the inner package is damaged and the pack is open


...

🧑 You:  thanks. bye


11:24:44 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
11:24:44 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
11:24:44 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
11:24:46 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler


🤖 Assistant: You’re welcome! 😊 If you ever need anything else or have more questions down the line, don’t hesitate to reach out. Take care and have a great day! Bye! 👋


🧑 You:  exit



🤖 Assistant: It was nice helping you! Goodbye! 👋


# ✅ Conclusion

This notebook demonstrates a realistic, CrewAI-powered multi-agent chat system.

Key features:
- Dynamic agent collaboration.
- Friendly, real-time conversation.
- Context retention across multiple user inputs.
- Conversation logging for records.